In [11]:
import networkx as nx
from collections import deque
import random

In [18]:
# compute a good p value
def good_p(k:int, G:nx.Graph):
  c_net = nx.transitivity(nx.Graph(G))
  return 1-((4 * c_net *(k-1))/(3*(k-2)))**(1/3)

def watts_strogatz(n:int, k:int, p:float):
  G = nx.MultiGraph(name="ws")
  
  # add n nodes
  for i in range(n):
    G.add_node(i)
  
  # create links in regular lattice
  for i in range(n):
    for j in range(i+1, i+1+k//2):
      # j is next nodes
      if(random.random() >= p):
        G.add_edge(i,j%n) # so that it loops around
      else:
        # rewire
        G.add_edge(i,random.randint(0, n-1)) # so that it loops around

  return G


In [19]:


def distance(G, i):
  D = [-1] * len(G) # D = {}
  Q = deque()
  D[i] = 0
  Q.append(i)
  while Q:
    i = Q.popleft()
    for j in G[i]:
      if D[j] == -1: # if j not in D:
        D[j] = D[i] + 1
        Q.append(j)
  return [d for d in D if d > 0]

def distances(G):
  D = []
  for i in random.sample(list(G.nodes()), 30):
    D.append(distance(G, i))
  return D

In [21]:
def info(G):
  print("{:>10s} | '{:s}'".format('Graph', G.name))

  n = G.number_of_nodes()
  n0, n1, delta = 0, 0, 0
  print("{:>10s} | {:,d} ({:,d}, {:,d})".format('Nodes', n, n0, n1))

  m = G.number_of_edges()
  print("{:>10s} | {:.2f} ({:,d})".format('Degree', 2 * m / n, delta))

  D = distances(G)
  D = [i for d in D for i in d]

  print("{:>10s} | {:.2f} ({:,d})".format('Distance', sum(D) / len(D), max(D)))

  avg_cluster = nx.average_clustering(nx.Graph(G))
  print(f"avg clustering: {avg_cluster}")

# we leave out the "www_google", since it is too big without improving the algorithm
# for name in ["karate_club","darknet",  "collaboration_imdb", "wikileaks", "enron", "www_google"]:
for name in ["darknet",  "collaboration_imdb", "wikileaks", "enron"]:

  G = nx.read_pajek(f"./{name}.net")
  G = nx.convert_node_labels_to_integers(G, label_attribute = 'label')
  G.name = name
  info(G)

  # generate a graph
  n = len(G)
  m = G.number_of_edges()
  k = max(round(m/n)*2, 4) #atleast 4
  p = good_p(k, G)
  ws_graph = watts_strogatz(n, k, p)
  info(ws_graph)
  print("--------------------------------")

     Graph | 'darknet'
     Nodes | 7,178 (0, 0)
    Degree | 6.99 (0)
  Distance | 2.40 (5)
avg clustering: 0.7061111034602707
     Graph | 'ws'
     Nodes | 7,178 (0, 0)
    Degree | 6.00 (0)
  Distance | 5.23 (7)
avg clustering: 0.005429335068510345
--------------------------------
     Graph | 'collaboration_imdb'
     Nodes | 17,577 (0, 0)
    Degree | 32.66 (0)
  Distance | 4.96 (14)
avg clustering: 0.3390564076688012
     Graph | 'ws'
     Nodes | 17,577 (0, 0)
    Degree | 32.00 (0)
  Distance | 3.59 (5)
avg clustering: 0.3682746466026261
--------------------------------
     Graph | 'wikileaks'
     Nodes | 52,416 (0, 0)
    Degree | 3.00 (0)
  Distance | 6.42 (17)
avg clustering: 0.18641890687848792
     Graph | 'ws'
     Nodes | 52,416 (0, 0)
    Degree | 4.00 (0)
  Distance | 9.74 (14)
avg clustering: 0.13429696268238586
--------------------------------
     Graph | 'enron'
     Nodes | 87,273 (0, 0)
    Degree | 26.31 (0)
  Distance | 4.47 (16)
avg clustering: 0.1193422206

In [10]:
new_G = watts_strogatz(1000, 10, 0.05)
info(new_G)

     Graph | 'ws'
     Nodes | 1,000 (0, 0)
    Degree | 10.00 (0)
  Distance | 5.19 (9)
avg clustering: 0.5761948051948037


## Scale free graphs

In [22]:
def price_graph(n:int, c:int, a:float):
  G = nx.MultiDiGraph(name="price graph")
  G.add_node(0)
  Q = [] # as many occurances as in degree
  # star graph
  for i in range(1,c+1):
    G.add_node(i)
    G.add_edge(0,i)
    Q.append(i)

  for i in range(c+1, n):
    G.add_node(i)
    for _ in range(c):
      # create links; select node j
      # choose uniform or list sampling
      if(random.random() < c/(c+a)):
        # sample random element from Q
        j = random.choice(Q)
      else:
        j = random.randint(0, i-1)
      
      G.add_edge(i,j)
      Q.append(j) #update in degrees

  return G

In [24]:
pg = price_graph(10000, 5, 3)
info(pg)

     Graph | 'price graph'
     Nodes | 10,000 (0, 0)
    Degree | 9.99 (0)
  Distance | 4.86 (13)
avg clustering: 0.027294911494265432
